In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install evaluate

In [4]:
from huggingface_hub import login, notebook_login
from transformers import AutoTokenizer
import transformers
import torch
import evaluate

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `AutoGen` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `AutoGen`


In [7]:
from transformers import DistilBertTokenizerFast, AutoTokenizer, AutoModelForCausalLM
from transformers import DistilBertForQuestionAnswering, AutoModelForQuestionAnswering

In [8]:
# DistilBERT base model (uncased)
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def QAinference(model, tokenizer, question, context, device='cpu'):
  inputs = tokenizer(question, context, return_tensors="pt")
  inputs = {k: v.to(device) for k, v in inputs.items()}
  with torch.no_grad():
    outputs = model(**inputs)
  #Get the highest probability from the model output for the start and end positions:
  absnswer_start_index = outputs.start_logits.argmax()
  answer_end_index = outputs.end_logits.argmax()
  #Decode the predicted tokens to get the answer:
  predict_answer_tokens = inputs['input_ids'][0, answer_start_index : answer_end_index + 1]
  answers=tokenizer.decode(predict_answer_tokens)
  print(answers)
  return answers

In [10]:
model.to(device)

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
     

In [11]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."
inputs = tokenizer(question, context, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}
with torch.no_grad():
  outputs = model(**inputs)
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ 0.1858, -0.0406, -0.5753, -0.1145, -0.4018, -0.5013, -0.3227, -0.2474,
         -0.3795,  0.0416, -0.2853, -0.5553, -0.4230, -0.4132, -0.3388, -0.3830,
         -0.4982, -0.2412, -0.2629, -0.3116, -0.2799, -0.3544, -0.2106, -0.2964,
         -0.4787, -0.0597, -0.1354, -0.3287,  0.2208,  0.0418]],
       device='cuda:0'), end_logits=tensor([[-0.1583, -0.3158, -0.3233, -0.6613, -0.5301, -0.2522, -0.4168, -0.4606,
         -0.5592,  0.3961, -0.3200, -0.5158, -0.1248, -0.1638, -0.1555, -0.0949,
         -0.3862, -0.3478, -0.2141, -0.2611, -0.2385, -0.3626, -0.3590, -0.4752,
          0.0351, -0.2354, -0.6623, -0.4849,  0.4811,  0.3962]],
       device='cuda:0'), hidden_states=None, attentions=None)

In [12]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [13]:
#Decode the predicted tokens to get the answer
predict_answer_tokens = inputs['input_ids'][0, answer_start_index : answer_end_index + 1]
answers=tokenizer.decode(predict_answer_tokens)
print(answers)

.


In [14]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."
answers=QAinference(model, tokenizer, question, context,device)

.


In [15]:
metric = evaluate.load("squad")

In [16]:
from transformers import DefaultDataCollator
from torch.utils.data import DataLoader

In [ ]:
!pip install datasets

In [18]:
from datasets import load_dataset

In [19]:
raw_datasets = load_dataset("squad")

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [20]:
max_length = 384
stride = 128
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]] #100 questions
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")#100, if no overflow, then sample_map=0-99
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx]) #get example id strings

        sequence_ids = inputs.sequence_ids(i) #[None, 0... None, 1... 1]
        offset = inputs["offset_mapping"][i] #100 size array of tuple (0, 4)
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ] #put None in sequence_id==1, i.e., put questions to None

    inputs["example_id"] = example_ids #string list
    return inputs

In [21]:
valkeyname="validation"
small_eval_set = raw_datasets[valkeyname].select(range(100))

In [22]:
validation_dataset = small_eval_set.map(
            preprocess_validation_examples, #preprocess_function, #preprocess_validation_examples,
            batched=True,
            remove_columns=raw_datasets[valkeyname].column_names,
        )

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [23]:
print(len(raw_datasets[valkeyname]))
print(len(validation_dataset))

10570
100


In [24]:
eval_set_for_model = validation_dataset.remove_columns(["example_id", "offset_mapping"])

In [25]:
print(validation_dataset.features.keys())#['input_ids', 'attention_mask', 'offset_mapping', 'example_id']
print(eval_set_for_model.features.keys())#['input_ids', 'attention_mask']

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'example_id'])
dict_keys(['input_ids', 'attention_mask'])


In [26]:
eval_set_for_model.set_format("torch")

In [27]:
batch_size =32
data_collator = DefaultDataCollator()
eval_dataloader = DataLoader(
  eval_set_for_model, batch_size=batch_size, collate_fn=data_collator
    )

In [28]:
num_val_steps = len(eval_dataloader)
start_logits = []
end_logits = []

In [29]:
from tqdm import tqdm

In [30]:
for batch in eval_dataloader:
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
  start_logits.append(outputs.start_logits.cpu().numpy())
  end_logits.append(outputs.end_logits.cpu().numpy())

In [31]:
def compute_metrics(start_logits, end_logits, features, examples):
    n_best = 20
    max_answer_length = 30

    #features is after tokenization, examples are original dataset
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [32]:
import numpy as np
import collections

In [33]:
# Evalation with 100 smapels
start_logits = np.concatenate(start_logits)
end_logits = np.concatenate(end_logits)
dataset_len=len(validation_dataset)
start_logits = start_logits[: dataset_len]
end_logits = end_logits[: dataset_len]
metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets[valkeyname]
    )
print(metrics)

100%|██████████| 10570/10570 [00:00<00:00, 12267.21it/s]


{'exact_match': 0.02838221381267739, 'f1': 0.039761937631304055}


In [34]:
print(metrics)

{'exact_match': 0.02838221381267739, 'f1': 0.039761937631304055}


In [35]:
max_length = 384
stride = 128
def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping") #new add
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i] #new add
        #answer = answers[i]
        answer = answers[sample_idx] # sample_idx from sample_map
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        # if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [36]:
tokenized_datasets = {}
tokenized_datasets["train"] = raw_datasets["train"].map(preprocess_training_examples, batched=True, remove_columns=raw_datasets["train"].column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

In [44]:
from torch.utils.data import DataLoader, random_split
#train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=batch_size, shuffle=True, collate_fn=data_collator)
# Parameters
train_ratio = 0.8  # 80% training, 20% validation
batch_size = 64

# Original dataset
dataset = tokenized_datasets["train"]

# Split dataset
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(f"Training size: {len(train_dataset)}, Validation size: {len(val_dataset)}")

# DataLoaders
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator
)

Training size: 70819, Validation size: 17705


In [52]:
from transformers import AdamW, get_scheduler
learningrate = 0.001
total_epochs = 2
optimizer = AdamW(model.parameters(), lr=learningrate)
num_epochs = total_epochs
num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
            "linear",
            optimizer=optimizer,
            num_warmup_steps=0,
            num_training_steps=num_training_steps,
        )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [40]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [45]:
model_save_path = "/content/drive/MyDrive/SJSU_Fall_2024/CMPE_258_Deep_Learning/HomeWork/HomeWork_2/model/model_trained.pth"
history_save_path = "/content/drive/MyDrive/SJSU_Fall_2024/CMPE_258_Deep_Learning/HomeWork/HomeWork_2/model/history.txt"
# Initialize tracking variables
training_history = {"epoch": [], "train_loss": [], "val_loss": []}

In [54]:
for epoch in tqdm(range(num_epochs)):
  # Training
  model.train()
  for batch in train_dataloader:
    optimizer.zero_grad()
    #batch = {k: v.to(device) for k, v in batch.items()}
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    #outputs = model(**batch)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    #sequence classification: outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss = outputs[0] #same loss results
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
  # Average training loss
  avg_train_loss = loss / len(train_dataloader)
  print(f"Epoch {epoch}, Average Training Loss: {avg_train_loss:.4f}")

  # Validation
  model.eval()
  total_val_loss = 0.0
  with torch.no_grad():
    for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            total_val_loss += outputs.loss.item()

    # Average validation loss
    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Epoch {epoch}, Validation Loss: {avg_val_loss:.4f}")

    # Save training and validation loss
    training_history["epoch"].append(epoch)
    training_history["train_loss"].append(avg_train_loss)
    training_history["val_loss"].append(avg_val_loss)

  # Save the model
  torch.save(model.state_dict(), model_save_path)

# Save the training history
with open(history_save_path, "w") as f:
  f.write("Epoch\tTrain Loss\tValidation Loss\n")
  for epoch, train_loss, val_loss in zip(training_history["epoch"],
                                          training_history["train_loss"],
                                          training_history["val_loss"]):
    f.write(f"{epoch}\t{train_loss:.4f}\t{val_loss:.4f}\n")

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 0, Average Training Loss: 0.0054
Epoch 0, Validation Loss: 5.9506


 50%|█████     | 1/2 [39:28<39:28, 2368.72s/it]

Epoch 1, Average Training Loss: 0.0054
Epoch 1, Validation Loss: 5.9506


100%|██████████| 2/2 [1:18:57<00:00, 2368.89s/it]


In [55]:
batch_size =32
data_collator = DefaultDataCollator()
eval_dataloader = DataLoader(
  eval_set_for_model, batch_size=batch_size, collate_fn=data_collator
    )

In [56]:
num_val_steps = len(eval_dataloader)
start_logits = []
end_logits = []

In [57]:
for batch in eval_dataloader:
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
  start_logits.append(outputs.start_logits.cpu().numpy())
  end_logits.append(outputs.end_logits.cpu().numpy())

In [58]:
# Evalation with 100 smapels
start_logits = np.concatenate(start_logits)
end_logits = np.concatenate(end_logits)
dataset_len=len(validation_dataset)
start_logits = start_logits[: dataset_len]
end_logits = end_logits[: dataset_len]
metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets[valkeyname]
    )
print(metrics)

100%|██████████| 10570/10570 [00:00<00:00, 13034.56it/s]


{'exact_match': 0.02838221381267739, 'f1': 0.03606646383253896}
